In [1]:
import os 
import pandas as pd 
import sys 
import re 

current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path,'..','..'))  # '..','..','..'
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
    
from pipeline.plotting.TS_analysis import plot_TS
from jupyter_ipynb.NetMob_training_analysis.plotting import plot_boxplot_on_metric
from experiences.convert_df_to_latex import dataframe_to_latex,parse_index_exp1_2,parse_index_exp4
from experiences.convert_df_to_latex import update_df_metrics,load_csv
from experiences.convert_df_to_latex import tackle_trial_j as tackle_trial_for_distrib
from experiences.contextual_data_integration.exp4_15min_h1_results import results as results_Exp4_15min_h1
from experiences.contextual_data_integration.exp4_15min_results import results  as results_Exp4_15min
from experiences.contextual_data_integration.exp4_results import results  as results_Exp4
from experiences.contextual_data_integration.exp1_subway_in_results import results  as results_Exp1_subway_in
from experiences.common_results import dic_exp_to_names,find_baseline,dic_trials

folder_path = '../../save/K_fold_validation/training_wo_HP_tuning'  # '../../save'


In [2]:
for exp_i,trials in dic_trials.items():
    if exp_i == 'set_experience_HERE' :#'Exp1_subway_in': #'set_experience_HERE': # set_experience_HERE # 'Exp1_subway_in'# 'Exp4_15min' # 'Exp4' # 'Exp3_bike_15min_h4' # 'Exp3' # 'Exp1'
        for trial_j in trials:
            df_j_all = pd.DataFrame()
            for n_bis in range(1,6):
                file_path = f"{folder_path}/{exp_i}/{dic_exp_to_names[exp_i]}/{trial_j}_bis{n_bis}/Losses_{trial_j}_bis{n_bis}.csv"
                df_j = pd.read_csv(file_path,index_col = 0)
                df_j.columns = [f'Train_{n_bis}',f'Valid_{n_bis}']
                df_j_all = pd.concat([df_j_all,df_j],axis=1)
            plot_TS(df_j_all,width=1500,height=400,bool_show=True,title=f"{exp_i}, {trial_j}",x_datetime = False)


In [3]:
import os 
import pandas as pd 
from pipeline.plotting.TS_analysis import plot_TS

def tackle_trial_j(folder_path,dic_exp_to_names,L_metrics,exp_i,trial_j,metrics,plot_losses):
    df_j_all = pd.DataFrame()
    metric_i = []
    for n_bis in range(1,6):
        df_j_all, metric_i = load_csv(folder_path,dic_exp_to_names,exp_i,trial_j,n_bis,df_j_all,metric_i,metrics)

    metric_i = pd.DataFrame(pd.DataFrame(metric_i).agg(['mean','std']).unstack()).T
    metric_i.index = [f"{trial_j}"]
    L_metrics.append(metric_i)
    if plot_losses:
            plot_TS(df_j_all,width=1500,height=400,bool_show=True,title=f"{exp_i}, {trial_j}",x_datetime = False)
    return L_metrics

folder_path = '../../save/K_fold_validation/training_wo_HP_tuning'
metrics = ['rmse','mae','mase']
plot_losses = False 


for exp_i,trials in dic_trials.items():
    print(f"\n------------------ {exp_i} ---")
    L_metrics = []
    if exp_i == 'Exp4_15min_h1':
        exp_i_tmp = 'Exp4_15min'
    else:
        exp_i_tmp = exp_i

    for trial_j in trials:
        L_metrics = tackle_trial_j(folder_path,dic_exp_to_names,L_metrics,exp_i_tmp,trial_j,metrics,plot_losses)

    if exp_i == 'Exp2':
        exp_i_tmp = 'Exp3_bike_15min_h4'
        trial_j = 'STAEformer_bike_out_calendar__e200_h4'
        L_metrics = tackle_trial_j(folder_path,dic_exp_to_names,L_metrics,exp_i_tmp,trial_j,metrics,plot_losses)

    if exp_i == 'Exp3_bike_15min_h4':
        exp_i_tmp = 'Exp4_15min'
        trial_j = 'STAEformer_bike_out_calendar__e80_h4'
        L_metrics = tackle_trial_j(folder_path,dic_exp_to_names,L_metrics,exp_i_tmp,trial_j,metrics,plot_losses)
    df_metrics_all = pd.concat(L_metrics)

    horizons = list(set([c[0].split('_')[-1][1:] for c in df_metrics_all.columns]))
    for horizon in horizons:
        df_horizon = df_metrics_all[[c for c in df_metrics_all.columns if c[0].endswith(f"_h{horizon}")]].dropna()

        globals()[f"df_metrics_{exp_i}_h{horizon}"] = df_horizon.sort_values(by=[(f'rmse_h{horizon}','mean')])

        baseline_index = find_baseline(exp_i,h=horizon)
        for c in globals()[f"df_metrics_{exp_i}_h{horizon}"].columns:
            if 'mean' == c[1]:
                globals()[f"df_metrics_{exp_i}_h{horizon}"][c] = globals()[f"df_metrics_{exp_i}_h{horizon}"][c].apply(lambda x: f"{round(x,2)} ({round(100*(-1+x/globals()[f'df_metrics_{exp_i}_h{horizon}'].loc[baseline_index,c]),2)}\%)")
        
        display(globals()[f"df_metrics_{exp_i}_h{horizon}"])



------------------ Exp1_subway_in ---


rmse_h1            \
                                                               mean       std   
STAEformer_subway_in_subway_out_calendar_early_...  35.48 (-0.33\%)  0.135227   
STAEformer_subway_in_subway_out_calendar_late_f...  35.58 (-0.04\%)  0.059006   
STAEformer_subway_in_calendar__e80_h1                  35.6 (0.0\%)  0.228636   
STAEformer_subway_in_subway_out_calendar_early_...   35.62 (0.08\%)  0.178211   
STAEformer_subway_in_subway_out_calendar_late_f...   35.65 (0.15\%)  0.232848   
STAEformer_subway_in_subway_out_calendar_late_f...   35.77 (0.48\%)  0.198439   
STAEformer_subway_in_subway_out_calendar_early_...   35.78 (0.51\%)  0.203069   
STAEformer_subway_in_subway_out_calendar_early_...   35.78 (0.52\%)  0.326888   
STAEformer_subway_in_subway_out_calendar_late_f...   35.78 (0.52\%)  0.185853   
STAEformer_subway_in_subway_out_calendar_late_f...   35.83 (0.65\%)  0.258495   
STAEformer_subway_in_subway_out_calendar_late_f...   35.86 (0.73\%)  0.181671   
STAEformer_subway_in_subway_out_calendar_early_...    35.88 (0.8\%)  0.133729   
STAEformer_subway_in_subway_out_calendar_early_...   35.95 (0.98\%)  0.174882   
STAEformer_subway_in_subway_out_calendar_early_...    35.95 (1.0\%)  0.087887   
STAEformer_subway_in_subway_out_calendar_early_...   36.04 (1.26\%)  0.382746   

                                                             mae_h1            \
                                                               mean       std   
STAEformer_subway_in_subway_out_calendar_early_...  20.36 (-0.17\%)  0.033285   
STAEformer_subway_in_subway_out_calendar_late_f...  20.37 (-0.11\%)  0.045057   
STAEformer_subway_in_calendar__e80_h1                 20.39 (0.0\%)  0.154526   
STAEformer_subway_in_subway_out_calendar_early_...  20.38 (-0.08\%)  0.137855   
STAEformer_subway_in_subway_out_calendar_late_f...   20.49 (0.49\%)  0.160206   
STAEformer_subway_in_subway_out_calendar_late_f...   20.53 (0.65\%)  0.131922   
STAEformer_subway_in_subway_out_calendar_early_...  20.36 (-0.14\%)  0.092017   
STAEformer_subway_in_subway_out_calendar_early_...   20.49 (0.48\%)  0.155233   
STAEformer_subway_in_subway_out_calendar_late_f...   20.49 (0.48\%)  0.075744   
STAEformer_subway_in_subway_out_calendar_late_f...   20.51 (0.59\%)  0.129991   
STAEformer_subway_in_subway_out_calendar_late_f...    20.54 (0.7\%)  0.046128   
STAEformer_subway_in_subway_out_calendar_early_...   20.55 (0.78\%)  0.100583   
STAEformer_subway_in_subway_out_calendar_early_...   20.61 (1.04\%)  0.088788   
STAEformer_subway_in_subway_out_calendar_early_...    20.5 (0.51\%)  0.056700   
STAEformer_subway_in_subway_out_calendar_early_...   20.54 (0.72\%)  0.100518   

                                                           mase_h1            
                                                              mean       std  
STAEformer_subway_in_subway_out_calendar_early_...  0.61 (-0.17\%)  0.000994  
STAEformer_subway_in_subway_out_calendar_late_f...  0.61 (-0.11\%)  0.001350  
STAEformer_subway_in_calendar__e80_h1                 0.61 (0.0\%)  0.004623  
STAEformer_subway_in_subway_out_calendar_early_...  0.61 (-0.07\%)  0.004132  
STAEformer_subway_in_subway_out_calendar_late_f...   0.61 (0.49\%)  0.004799  
STAEformer_subway_in_subway_out_calendar_late_f...   0.61 (0.65\%)  0.003950  
STAEformer_subway_in_subway_out_calendar_early_...  0.61 (-0.14\%)  0.002758  
STAEformer_subway_in_subway_out_calendar_early_...   0.61 (0.48\%)  0.004646  
STAEformer_subway_in_subway_out_calendar_late_f...   0.61 (0.48\%)  0.002267  
STAEformer_subway_in_subway_out_calendar_late_f...   0.61 (0.59\%)  0.003896  
STAEformer_subway_in_subway_out_calendar_late_f...    0.61 (0.7\%)  0.001379  
STAEformer_subway_in_subway_out_calendar_early_...   0.62 (0.78\%)  0.003015  
STAEformer_subway_in_subway_out_calendar_early_...   0.62 (1.04\%)  0.002652  
STAEformer_subway_in_subway_out_calendar_early_...   0.61 (0.51\%)  0.001700  
STAEformer_subway_in_subway_out_calendar

rmse_h4            \
                                                               mean       std   
STAEformer_subway_in_subway_out_calendar_late_f...  44.09 (-1.23\%)  0.465219   
STAEformer_subway_in_subway_out_calendar_late_f...  44.17 (-1.05\%)  0.444862   
STAEformer_subway_in_subway_out_calendar_early_...  44.22 (-0.93\%)  0.452206   
STAEformer_subway_in_subway_out_calendar_late_f...  44.44 (-0.44\%)  0.391122   
STAEformer_subway_in_subway_out_calendar_early_...  44.57 (-0.16\%)  0.653885   
STAEformer_subway_in_calendar__e80_h4                 44.64 (0.0\%)  0.749028   
STAEformer_subway_in_subway_out_calendar_late_f...   44.64 (0.01\%)  0.311228   
STAEformer_subway_in_subway_out_calendar_early_...   44.68 (0.09\%)  0.449618   
STAEformer_subway_in_subway_out_calendar_early_...   44.71 (0.16\%)  0.616244   
STAEformer_subway_in_subway_out_calendar_early_...   44.85 (0.48\%)  0.511583   
STAEformer_subway_in_subway_out_calendar_early_...   44.87 (0.52\%)  0.475858   
STAEformer_subway_in_subway_out_calendar_late_f...   44.87 (0.53\%)  0.649576   
STAEformer_subway_in_subway_out_calendar_early_...    44.9 (0.58\%)  0.355340   
STAEformer_subway_in_subway_out_calendar_early_...   44.97 (0.76\%)  0.850792   
STAEformer_subway_in_subway_out_calendar_late_f...   45.33 (1.56\%)  0.969149   

                                                             mae_h4            \
                                                               mean       std   
STAEformer_subway_in_subway_out_calendar_late_f...  23.79 (-1.05\%)  0.297099   
STAEformer_subway_in_subway_out_calendar_late_f...  23.86 (-0.76\%)  0.273204   
STAEformer_subway_in_subway_out_calendar_early_...  23.84 (-0.82\%)  0.275521   
STAEformer_subway_in_subway_out_calendar_late_f...  23.96 (-0.31\%)  0.226310   
STAEformer_subway_in_subway_out_calendar_early_...  23.98 (-0.25\%)  0.399274   
STAEformer_subway_in_calendar__e80_h4                 24.04 (0.0\%)  0.235967   
STAEformer_subway_in_subway_out_calendar_late_f...  23.99 (-0.19\%)  0.129404   
STAEformer_subway_in_subway_out_calendar_early_...  23.91 (-0.55\%)  0.157793   
STAEformer_subway_in_subway_out_calendar_early_...  24.02 (-0.07\%)  0.256309   
STAEformer_subway_in_subway_out_calendar_early_...    24.04 (0.0\%)  0.300061   
STAEformer_subway_in_subway_out_calendar_early_...  24.03 (-0.05\%)  0.181445   
STAEformer_subway_in_subway_out_calendar_late_f...  23.98 (-0.25\%)  0.286581   
STAEformer_subway_in_subway_out_calendar_early_...    24.1 (0.27\%)  0.156948   
STAEformer_subway_in_subway_out_calendar_early_...    24.18 (0.6\%)  0.423145   
STAEformer_subway_in_subway_out_calendar_late_f...   24.25 (0.87\%)  0.429083   

                                                           mase_h4            
                                                              mean       std  
STAEformer_subway_in_subway_out_calendar_late_f...  0.71 (-1.05\%)  0.008906  
STAEformer_subway_in_subway_out_calendar_late_f...  0.71 (-0.76\%)  0.008177  
STAEformer_subway_in_subway_out_calendar_early_...  0.71 (-0.82\%)  0.008238  
STAEformer_subway_in_subway_out_calendar_late_f...  0.72 (-0.31\%)  0.006774  
STAEformer_subway_in_subway_out_calendar_early_...  0.72 (-0.25\%)  0.011965  
STAEformer_subway_in_calendar__e80_h4                 0.72 (0.0\%)  0.007073  
STAEformer_subway_in_subway_out_calendar_late_f...  0.72 (-0.19\%)  0.003872  
STAEformer_subway_in_subway_out_calendar_early_...  0.72 (-0.55\%)  0.004733  
STAEformer_subway_in_subway_out_calendar_early_...  0.72 (-0.07\%)  0.007685  
STAEformer_subway_in_subway_out_calendar_early_...   0.72 (-0.0\%)  0.008994  
STAEformer_subway_in_subway_out_calendar_early_...  0.72 (-0.05\%)  0.005423  
STAEformer_subway_in_subway_out_calendar_late_f...  0.72 (-0.25\%)  0.008583  
STAEformer_subway_in_subway_out_calendar_early_...   0.72 (0.27\%)  0.004703  
STAEformer_subway_in_subway_out_calendar_early_...    0.72 (0.6\%)  0.012667  
STAEformer_subway_in_subway_out_calendar


------------------ Exp1_subway_out ---


rmse_h1            \
                                                 mean       std   
STAEformer_subway_out_calendar__e80_h1  36.81 (0.0\%)  0.160002   

                                               mae_h1                 mase_h1  \
                                                 mean       std          mean   
STAEformer_subway_out_calendar__e80_h1  20.39 (0.0\%)  0.082622  0.63 (0.0\%)   

                                                  
                                             std  
STAEformer_subway_out_calendar__e80_h1  0.002557

...


------------------ Exp4_15min_h1 ---


rmse_h1            \
                                                              mean       std   
STAEformer_bike_out_subway_in_subway_out_calend...  4.29 (-0.21\%)  0.016560   
STAEformer_bike_out_calendar__e80_h1                   4.3 (0.0\%)  0.024679   
STAEformer_bike_out_subway_in_subway_out_calend...    4.3 (0.01\%)  0.014277   
STAEformer_bike_out_subway_in_subway_out_calend...    4.3 (0.03\%)  0.012854   
STAEformer_bike_out_subway_in_subway_out_calend...   4.31 (0.12\%)  0.013055   
STAEformer_bike_out_subway_in_subway_out_calend...   4.31 (0.15\%)  0.005196   
STAEformer_bike_out_subway_in_subway_out_calend...    4.31 (0.2\%)  0.020614   
STAEformer_bike_out_subway_in_subway_out_calend...   4.31 (0.22\%)  0.021746   
STAEformer_bike_out_subway_in_subway_out_calend...   4.31 (0.23\%)  0.025153   
STAEformer_bike_out_subway_in_subway_out_calend...   4.31 (0.28\%)  0.027600   
STAEformer_bike_out_subway_in_subway_out_calend...   4.31 (0.31\%)  0.025026   
STAEformer_bike_out_subway_in_subway_out_calend...   4.33 (0.63\%)  0.018610   
STAEformer_bike_out_subway_in_subway_out_calend...   4.34 (0.79\%)  0.031022   

                                                            mae_h1            \
                                                              mean       std   
STAEformer_bike_out_subway_in_subway_out_calend...  2.77 (-0.36\%)  0.003302   
STAEformer_bike_out_calendar__e80_h1                  2.78 (0.0\%)  0.016809   
STAEformer_bike_out_subway_in_subway_out_calend...  2.78 (-0.07\%)  0.016441   
STAEformer_bike_out_subway_in_subway_out_calend...   2.78 (-0.0\%)  0.020692   
STAEformer_bike_out_subway_in_subway_out_calend...  2.78 (-0.01\%)  0.009006   
STAEformer_bike_out_subway_in_subway_out_calend...   2.78 (0.04\%)  0.012281   
STAEformer_bike_out_subway_in_subway_out_calend...  2.78 (-0.08\%)  0.017602   
STAEformer_bike_out_subway_in_subway_out_calend...  2.78 (-0.08\%)  0.016412   
STAEformer_bike_out_subway_in_subway_out_calend...    2.78 (0.1\%)  0.017852   
STAEformer_bike_out_subway_in_subway_out_calend...   2.79 (0.15\%)  0.016595   
STAEformer_bike_out_subway_in_subway_out_calend...   2.78 (0.03\%)  0.010077   
STAEformer_bike_out_subway_in_subway_out_calend...    2.8 (0.62\%)  0.033335   
STAEformer_bike_out_subway_in_subway_out_calend...    2.8 (0.49\%)  0.017826   

                                                           mase_h1            
                                                              mean       std  
STAEformer_bike_out_subway_in_subway_out_calend...  0.72 (-0.36\%)  0.000859  
STAEformer_bike_out_calendar__e80_h1                  0.72 (0.0\%)  0.004360  
STAEformer_bike_out_subway_in_subway_out_calend...  0.72 (-0.07\%)  0.004266  
STAEformer_bike_out_subway_in_subway_out_calend...   0.72 (-0.0\%)  0.005372  
STAEformer_bike_out_subway_in_subway_out_calend...  0.72 (-0.01\%)  0.002334  
STAEformer_bike_out_subway_in_subway_out_calend...   0.72 (0.04\%)  0.003187  
STAEformer_bike_out_subway_in_subway_out_calend...  0.72 (-0.08\%)  0.004570  
STAEformer_bike_out_subway_in_subway_out_calend...  0.72 (-0.08\%)  0.004262  
STAEformer_bike_out_subway_in_subway_out_calend...    0.72 (0.1\%)  0.004629  
STAEformer_bike_out_subway_in_subway_out_calend...   0.72 (0.15\%)  0.004310  
STAEformer_bike_out_subway_in_subway_out_calend...   0.72 (0.03\%)  0.002620  
STAEformer_bike_out_subway_in_subway_out_calend...   0.73 (0.62\%)  0.008656  
STAEformer_bike_out_subway_in_subway_out_calend...   0.73 (0.49\%)  0.004629


------------------ Exp6_subway_netmob ---


rmse_h1            \
                                                               mean       std   
STAEformer_subway_in_netmob_POIs_calendar_late_...  35.74 (-1.71\%)  0.711533   
STAEformer_subway_in_netmob_POIs_calendar_late_...  35.93 (-1.18\%)  0.716963   
STAEformer_subway_in_netmob_POIs_calendar_late_...  35.94 (-1.16\%)  1.445708   
STAEformer_subway_in_netmob_POIs_calendar_late_...  36.03 (-0.91\%)  0.826459   
STAEformer_subway_in_netmob_POIs_calendar_late_...  36.22 (-0.37\%)  0.929231   
STAEformer_subway_in_netmob_POIs_calendar_late_...  36.29 (-0.19\%)  0.294127   
STAEformer_subway_in_calendar__e150_h1                36.36 (0.0\%)  0.416060   
STAEformer_subway_in_netmob_POIs_calendar_late_...   36.46 (0.28\%)  1.345812   
STAEformer_subway_in_netmob_POIs_calendar_late_...    36.6 (0.68\%)  0.759024   

                                                             mae_h1            \
                                                               mean       std   
STAEformer_subway_in_netmob_POIs_calendar_late_...   21.2 (-1.52\%)  0.300492   
STAEformer_subway_in_netmob_POIs_calendar_late_...  21.26 (-1.21\%)  0.249830   
STAEformer_subway_in_netmob_POIs_calendar_late_...  21.22 (-1.39\%)  0.729602   
STAEformer_subway_in_netmob_POIs_calendar_late_...  21.33 (-0.89\%)  0.503613   
STAEformer_subway_in_netmob_POIs_calendar_late_...  21.45 (-0.32\%)  0.587913   
STAEformer_subway_in_netmob_POIs_calendar_late_...    21.5 (-0.1\%)  0.212985   
STAEformer_subway_in_calendar__e150_h1                21.52 (0.0\%)  0.251448   
STAEformer_subway_in_netmob_POIs_calendar_late_...   21.54 (0.08\%)  0.688813   
STAEformer_subway_in_netmob_POIs_calendar_late_...   21.68 (0.71\%)  0.454884   

                                                           mase_h1            
                                                              mean       std  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.68 (-1.52\%)  0.009615  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.68 (-1.22\%)  0.007971  
STAEformer_subway_in_netmob_POIs_calendar_late_...   0.68 (-1.4\%)  0.023354  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.68 (-0.89\%)  0.016109  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.69 (-0.32\%)  0.018871  
STAEformer_subway_in_netmob_POIs_calendar_late_...   0.69 (-0.1\%)  0.006869  
STAEformer_subway_in_calendar__e150_h1                0.69 (0.0\%)  0.008054  
STAEformer_subway_in_netmob_POIs_calendar_late_...   0.69 (0.09\%)  0.022065  
STAEformer_subway_in_netmob_POIs_calendar_late_...   0.69 (0.71\%)  0.014583

rmse_h4            \
                                                              mean       std   
STAEformer_subway_in_calendar__e150_h4               43.73 (0.0\%)  2.225578   
STAEformer_subway_in_netmob_POIs_calendar_late_...  44.25 (1.19\%)  2.999800   
STAEformer_subway_in_netmob_POIs_calendar_late_...  44.83 (2.51\%)  2.316863   
STAEformer_subway_in_netmob_POIs_calendar_late_...  45.37 (3.76\%)  0.988276   
STAEformer_subway_in_netmob_POIs_calendar_late_...  45.66 (4.42\%)  1.828776   
STAEformer_subway_in_netmob_POIs_calendar_late_...  45.92 (5.01\%)  2.109974   
STAEformer_subway_in_netmob_POIs_calendar_late_...   46.05 (5.3\%)  2.295588   
STAEformer_subway_in_netmob_POIs_calendar_late_...   46.09 (5.4\%)  2.408815   
STAEformer_subway_in_netmob_POIs_calendar_late_...  46.16 (5.56\%)  3.189010   

                                                            mae_h4            \
                                                              mean       std   
STAEformer_subway_in_calendar__e150_h4               25.33 (0.0\%)  1.145708   
STAEformer_subway_in_netmob_POIs_calendar_late_...  25.48 (0.57\%)  1.052312   
STAEformer_subway_in_netmob_POIs_calendar_late_...  25.41 (0.32\%)  0.930834   
STAEformer_subway_in_netmob_POIs_calendar_late_...  25.86 (2.11\%)  0.226085   
STAEformer_subway_in_netmob_POIs_calendar_late_...   25.79 (1.8\%)  0.771501   
STAEformer_subway_in_netmob_POIs_calendar_late_...  25.91 (2.28\%)  0.686228   
STAEformer_subway_in_netmob_POIs_calendar_late_...   25.96 (2.5\%)  0.917877   
STAEformer_subway_in_netmob_POIs_calendar_late_...  26.37 (4.12\%)  1.016179   
STAEformer_subway_in_netmob_POIs_calendar_late_...  26.21 (3.46\%)  0.936478   

                                                          mase_h4            
                                                             mean       std  
STAEformer_subway_in_calendar__e150_h4               0.81 (0.0\%)  0.036620  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.82 (0.57\%)  0.033686  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.81 (0.32\%)  0.029798  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.83 (2.11\%)  0.007292  
STAEformer_subway_in_netmob_POIs_calendar_late_...   0.83 (1.8\%)  0.024721  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.83 (2.28\%)  0.021967  
STAEformer_subway_in_netmob_POIs_calendar_late_...   0.83 (2.5\%)  0.029370  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.84 (4.12\%)  0.032476  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.84 (3.46\%)  0.029944

In [7]:
exp_i = 'Exp1_subway_in' #'Exp1_subway_in' # 'Exp1' #'Exp4_15min', 'Exp4' #'Exp3_bike_15min_h4' #'Exp3' #'Exp1'
metrics = ['rmse','mae','mase']
trials = dic_trials[exp_i]
print(f"\n------------------ {exp_i} ---")

L_metrics = []
for trial_j in trials:
    L_metrics = tackle_trial_for_distrib(folder_path,dic_exp_to_names,L_metrics,exp_i,trial_j,metrics)

if exp_i == 'Exp4':
    exp_i_tmp = 'Exp3'
    trial_j = 'STAEformer_bike_out_calendar__e200_h1'
    L_metrics = tackle_trial_for_distrib(folder_path,dic_exp_to_names,L_metrics,exp_i_tmp,trial_j,metrics)
if exp_i == 'Exp4_15min':
    exp_i_tmp = 'Exp3_bike_15min_h4'
    trial_j = 'STAEformer_bike_out_calendar__e200_h4'
    L_metrics = tackle_trial_for_distrib(folder_path,dic_exp_to_names,L_metrics,exp_i_tmp,trial_j,metrics)
    
df_metrics_all = pd.concat(L_metrics)

horizons = list(set([c.split('_')[-1][1:] for c in df_metrics_all.columns]))
for horizon in horizons:
    print('\n----------------')
    print(f"Horizon: {horizon}")
    df_horizon = df_metrics_all[[c for c in df_metrics_all.columns if c.endswith(f"_h{horizon}")]].dropna()

    df_horizon = update_df_metrics(df_horizon,exp_i)

    plot_boxplot_on_metric(df_horizon, metric_i='mae', xaxis_label="Config", legend_group='legend_group', width=1000, height=400, 
                                save_path=None)
    plot_boxplot_on_metric(df_horizon, metric_i='rmse', xaxis_label="Config", legend_group='legend_group', width=1600, height=800, 
                                save_path=None)
    # plot_boxplot_on_metric(df_horizon, metric_i='mase', xaxis_label="Config", legend_group='legend_group', width=1200, height=400, 
    #                             save_path=None)



------------------ Exp1_subway_in ---

----------------
Horizon: 1


Loading BokehJS ...

Loading BokehJS ...


----------------
Horizon: 4


Loading BokehJS ...

Loading BokehJS ...

In [19]:
df_metrics_Exp1_subway_in_h1
df_metrics_Exp1_subway_in_h1

rmse_h1            \
                                                               mean       std   
STAEformer_subway_in_subway_out_calendar_early_...  35.48 (-0.33\%)  0.135227   
STAEformer_subway_in_subway_out_calendar_late_f...  35.58 (-0.04\%)  0.059006   
STAEformer_subway_in_calendar__e80_h1                  35.6 (0.0\%)  0.228636   
STAEformer_subway_in_subway_out_calendar_early_...   35.62 (0.08\%)  0.178211   
STAEformer_subway_in_subway_out_calendar_late_f...   35.65 (0.15\%)  0.232848   
STAEformer_subway_in_subway_out_calendar_late_f...   35.77 (0.48\%)  0.198439   
STAEformer_subway_in_subway_out_calendar_early_...   35.78 (0.51\%)  0.203069   
STAEformer_subway_in_subway_out_calendar_early_...   35.78 (0.52\%)  0.326888   
STAEformer_subway_in_subway_out_calendar_late_f...   35.78 (0.52\%)  0.185853   
STAEformer_subway_in_subway_out_calendar_late_f...   35.83 (0.65\%)  0.258495   
STAEformer_subway_in_subway_out_calendar_late_f...   35.86 (0.73\%)  0.181671   
STAEformer_subway_in_subway_out_calendar_early_...    35.88 (0.8\%)  0.133729   
STAEformer_subway_in_subway_out_calendar_early_...   35.95 (0.98\%)  0.174882   
STAEformer_subway_in_subway_out_calendar_early_...    35.95 (1.0\%)  0.087887   
STAEformer_subway_in_subway_out_calendar_early_...   36.04 (1.26\%)  0.382746   

                                                             mae_h1            \
                                                               mean       std   
STAEformer_subway_in_subway_out_calendar_early_...  20.36 (-0.17\%)  0.033285   
STAEformer_subway_in_subway_out_calendar_late_f...  20.37 (-0.11\%)  0.045057   
STAEformer_subway_in_calendar__e80_h1                 20.39 (0.0\%)  0.154526   
STAEformer_subway_in_subway_out_calendar_early_...  20.38 (-0.08\%)  0.137855   
STAEformer_subway_in_subway_out_calendar_late_f...   20.49 (0.49\%)  0.160206   
STAEformer_subway_in_subway_out_calendar_late_f...   20.53 (0.65\%)  0.131922   
STAEformer_subway_in_subway_out_calendar_early_...  20.36 (-0.14\%)  0.092017   
STAEformer_subway_in_subway_out_calendar_early_...   20.49 (0.48\%)  0.155233   
STAEformer_subway_in_subway_out_calendar_late_f...   20.49 (0.48\%)  0.075744   
STAEformer_subway_in_subway_out_calendar_late_f...   20.51 (0.59\%)  0.129991   
STAEformer_subway_in_subway_out_calendar_late_f...    20.54 (0.7\%)  0.046128   
STAEformer_subway_in_subway_out_calendar_early_...   20.55 (0.78\%)  0.100583   
STAEformer_subway_in_subway_out_calendar_early_...   20.61 (1.04\%)  0.088788   
STAEformer_subway_in_subway_out_calendar_early_...    20.5 (0.51\%)  0.056700   
STAEformer_subway_in_subway_out_calendar_early_...   20.54 (0.72\%)  0.100518   

                                                           mase_h1            
                                                              mean       std  
STAEformer_subway_in_subway_out_calendar_early_...  0.61 (-0.17\%)  0.000994  
STAEformer_subway_in_subway_out_calendar_late_f...  0.61 (-0.11\%)  0.001350  
STAEformer_subway_in_calendar__e80_h1                 0.61 (0.0\%)  0.004623  
STAEformer_subway_in_subway_out_calendar_early_...  0.61 (-0.07\%)  0.004132  
STAEformer_subway_in_subway_out_calendar_late_f...   0.61 (0.49\%)  0.004799  
STAEformer_subway_in_subway_out_calendar_late_f...   0.61 (0.65\%)  0.003950  
STAEformer_subway_in_subway_out_calendar_early_...  0.61 (-0.14\%)  0.002758  
STAEformer_subway_in_subway_out_calendar_early_...   0.61 (0.48\%)  0.004646  
STAEformer_subway_in_subway_out_calendar_late_f...   0.61 (0.48\%)  0.002267  
STAEformer_subway_in_subway_out_calendar_late_f...   0.61 (0.59\%)  0.003896  
STAEformer_subway_in_subway_out_calendar_late_f...    0.61 (0.7\%)  0.001379  
STAEformer_subway_in_subway_out_calendar_early_...   0.62 (0.78\%)  0.003015  
STAEformer_subway_in_subway_out_calendar_early_...   0.62 (1.04\%)  0.002652  
STAEformer_subway_in_subway_out_calendar_early_...   0.61 (0.51\%)  0.001700  
STAEformer_subway_in_subway_out_calendar

In [ ]:
# --- Captions Dictionary ---
captions = {
    "Exp1": r"Prediction of Subway-Out with STAEformer using contextual data 'Subway-In' according to different Integration Strategies. Metrics are averaged over 5 runs.",
    "Exp1_subway_in": r"Prediction of Subway-In with STAEformer using contextual data 'Subway-Out' according to different Integration Strategies. Metrics are averaged over 5 runs.",
    "Exp2": r"Prediction of Bike-Out (15min aggregation) at a 60-minute horizon with STAEformer, using weather forecasts as contextual data. Metrics are averaged over 5 runs.",
    "Exp4": r"Prediction of Bike-Out with STAEformer using heterogenous contextual data 'subway-in-subway-out' (1h aggregation) at horizon +60min. The table compares the baseline with different configurations of the CrossAttnBackBone model. Metrics are averaged over 5 runs.",
    "Exp4_15min": r"Prediction of Bike-Out with STAEformer using heterogenous contextual data 'subway-in-subway-out' (15min aggregation) at horizon +60min. This table compares various CrossAttnBackBone model configurations. Metrics are averaged over 5 runs.",
     "Exp4_15min_h1": r"Prediction of Bike-Out with STAEformer using heterogenous contextual data 'subway-in-subway-out' (15min aggregation) at horizon +15min. This table compares various CrossAttnBackBone model configurations. Metrics are averaged over 5 runs.",
}

# --- Generate and Print LaTeX Tables ---
print("%%% LaTeX code for Experiment 1 Table %%%\n")
latex_table_1 = dataframe_to_latex(df_metrics_Exp1_h4, captions["Exp1"], "exp1_subway_context", parse_index_exp1_2)
print(latex_table_1)

print("\n" + "="*80 + "\n")

print("\n------------------------------------------------\n%%% LaTeX code for Experiment 1 Horizon +15min Table %%%\n")
latex_table_1 = dataframe_to_latex(df_metrics_Exp1_subway_in_h1, captions["Exp1_subway_in"], "exp1_subway_context", lambda x: parse_index_exp1_2(x, contextual='subway_out'))
print(latex_table_1)

print("\n" + "="*80 + "\n")

print("\n------------------------------------------------\n%%% LaTeX code for Experiment 1 Horizon +60min Table %%%\n")
latex_table_1 = dataframe_to_latex(df_metrics_Exp1_subway_in_h4, captions["Exp1_subway_in"], "exp1_subway_context", lambda x: parse_index_exp1_2(x, contextual='subway_out'))
print(latex_table_1)

print("\n" + "="*80 + "\n")

print("\n------------------------------------------------\n%%% LaTeX code for Experiment 2 Table %%%\n")
# Note: parse_index_exp1_2 is reused as it correctly handles the index format for Exp2
latex_table_2 = dataframe_to_latex(df_metrics_Exp2_h4, captions["Exp2"], "exp2_bike_weather", parse_index_exp1_2)
print(latex_table_2)

print("\n" + "="*80 + "\n")

print("\n------------------------------------------------\n%%% LaTeX code for Experiment 4 Table %%%\n")
latex_table_4 = dataframe_to_latex(df_metrics_Exp4_h1, captions["Exp4"], "exp4_bike_hetero", parse_index_exp4)
print(latex_table_4)

print("\n" + "="*80 + "\n")

print("\n------------------------------------------------\n%%% LaTeX code for Experiment 4 (15min) at horizon +60min Table %%%\n")
# Note: parse_index_exp4 is reused as it correctly handles the index format for Exp4_15min
latex_table_4_15min = dataframe_to_latex(df_metrics_Exp4_15min_h4, captions["Exp4_15min"], "exp4_bike_hetero_15min", parse_index_exp4)
print(latex_table_4_15min)

print("\n" + "="*80 + "\n")

print("\n------------------------------------------------\n%%% LaTeX code for Experiment 4 (agg 15min) at horizon + 15min Table %%%\n")
latex_table_4_15min_h1 = dataframe_to_latex(df_metrics_Exp4_15min_h4, captions["Exp4_15min_h1"], "exp4_bike_hetero_15min", parse_index_exp4)
print(latex_table_4_15min_h1)

print("\n" + "="*80 + "\n")


%%% LaTeX code for Experiment 1 Table %%%

\begin{table}[!htb]
    \centering
    \caption{Prediction of Subway-Out with STAEformer using contextual data 'Subway-In' according to different Integration Strategies. Metrics are averaged over 5 runs.}
    \label{tab:exp1_subway_context}
    \resizebox{\textwidth}{!}{
    \begin{tabular}{{llcccccc}}
        \toprule
        Contextual Data & Integration Strategy & \multicolumn{2}{c}{ RMSE (h4) } & \multicolumn{2}{c}{ MAE (h4) } & \multicolumn{2}{c}{ MASE (h4) } \\
        \cmidrule(lr){1-2} \cmidrule(lr){3-4} \cmidrule(lr){5-6} \cmidrule(lr){7-8}
         &  & Mean & Std & Mean & Std & Mean & Std \\
        \midrule
        Yes & Early Fusion Feature Extractor & 40.46 (-5.3\%) & 0.1824 & 22.27 (-4.02\%) & 0.0953 & 0.69 (-3.53\%) & 0.0030 \\
        Yes & Early Fusion Shared Embedding & 40.76 (-4.61\%) & 0.4171 & 22.3 (-3.88\%) & 0.2049 & 0.69 (-3.38\%) & 0.0064 \\
        Yes & Late Fusion Traffic Model Backbone & 40.8 (-4.52\%) & 0.2193 & 

In [22]:

import os 
import sys 

current_path = os.path.abspath(os.getcwd())
parent_dir = os.path.abspath(os.path.join(current_path, '..','..','..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.accuracy_comparison import load_trainer_ds_from_2_trials,get_predict_real_and_inputs,get_model_args
from examples.accuracy_comparison import get_rainy_indices,plot_analysis_comparison_2_config,print_global_info,get_previous_and_prediction,get_gain_from_mod1
import torch 
from pipeline.utils.metrics import evaluate_metrics

def get_dict_metrics_on_rainy_events(full_predict1,full_predict2,Y_true,X,args_init1,args_init2,ds2):
    h_idx = 1
    metric_list = ['rmse','mse','mae','mase','mape']
    previous,_,_,_ = get_previous_and_prediction(full_predict1,full_predict2,Y_true,X,h_idx)
    assert args_init1.horizon_step == args_init1.step_ahead, "Horizon step must be equal to step_ahead here"

    print("\nComparison on between models across all time-slots followed by comparison on Rainy Events Only")
    _,train_rainy_indices,_ = get_rainy_indices(args = args_init2,ds = ds2,training_mode = 'train')
    print(f"Number of rainy time-slots in the train set: {len(train_rainy_indices)}, i.e {len(train_rainy_indices)/len(ds2.tensor_limits_keeper.df_verif_train)*100:.2f} % of the train set")
    # ---- Plot Accuracy comparison on rainy moments only ----
    mask,rainy_indices,df_weather = get_rainy_indices(args = args_init2,ds = ds2,training_mode = 'test')
    print(f"Number of rainy time-slots in the test set: {len(rainy_indices)}, i.e {len(rainy_indices)/len(ds2.tensor_limits_keeper.df_verif_test)*100:.2f} % of the test set\n")

    dates = mask[mask].index
    masked_index = mask.reset_index(drop=True)
    masked_index = masked_index[masked_index].index


    # --- Get Prediction on rainy time-slots:
    rainy_predict1 = torch.index_select(full_predict1,0,torch.tensor(masked_index).long())
    rainy_predict2 = torch.index_select(full_predict2,0,torch.tensor(masked_index).long())
    rainy_Y_true = torch.index_select(Y_true,0,torch.tensor(masked_index).long())
    rainy_previous = torch.index_select(previous,0,torch.tensor(masked_index).long())

    dic_metric1 = evaluate_metrics(rainy_predict1,rainy_Y_true,metrics = metric_list, previous = rainy_previous,horizon_step = h_idx)
    dic_metric1 = dict(dic_metric1.copy())
    dic_metric2 = evaluate_metrics(rainy_predict2,rainy_Y_true,metrics = metric_list, previous = rainy_previous,horizon_step = h_idx)
    dic_metric2 = dict(dic_metric2.copy())
    return dic_metric1,dic_metric2



dic_contextual_data = {'bike_out': [[]],  #,['weather','calendar']
                    }

dic_fusion_strategie = {('bike_out',()): [''],
                        ('bike_out',('weather','calendar')):['early_fusion','late_fusion'],
                        }
             
dic_feature_extractor = {('bike_out',()): [''],
                        ('bike_out',('weather','calendar')):['repeat_t_proj','feature_extractor']
                        }


model_name = 'STAEformer'
epochs = 200
range_k = range(1,6)
training_mode = 'test'
modification = {'shuffle':False,
                'data_augmentation':False,
                'torch_compile': False,
                }
log = ''
Exp = 'Exp2'  # 'Exp2'

for horizon in [4]:
    for target in ['bike_out']:
        for contextual_data in dic_contextual_data[target]:
            reversed_metric = False
            fusion_strategies = dic_fusion_strategie[(target,tuple(contextual_data))]
            for fusion_strategie in fusion_strategies:
                feature_extractors = dic_feature_extractor[(target,tuple(contextual_data))]
                for feature_extractor in feature_extractors:
                    if not('weather' in contextual_data):
                        print('\n------------------------------------------------------\nContextual data:\n')
                        contextual_data1 = ['weather','calendar'] if target == 'bike_out' else ['subway_in','weather','calendar']
                        fusion_strategie1 = dic_fusion_strategie[(target,tuple(contextual_data1))][0]
                        feature_extractor1 = dic_feature_extractor[(target,tuple(contextual_data1))][0]

                        trial_id1 = f"{model_name}_{target}_calendar__e{epochs}_h{horizon}_bis"
                        trial_id2 = f"{model_name}_{target}_{'_'.join(contextual_data1)}_{fusion_strategie1}_{feature_extractor1}__e{epochs}_h{horizon}_bis"
                        reversed_metric = True 
                    else:
                        trial_id1 = f"{model_name}_{target}_calendar__e{epochs}_h{horizon}_bis"
                        trial_id2 = f"{model_name}_{target}_{'_'.join(contextual_data)}_{fusion_strategie}_{feature_extractor}__e{epochs}_h{horizon}_bis"

                    model_args,_,path_model_args,_ = get_model_args(target,model_name,save_folder_name = f'{Exp}/{target}_{model_name}')
                    ds1,ds2,args_init1,args_init2 = None, None, None, None
                    for k in range_k:
                        trial_id1_updated = f"{trial_id1}{k}_f5"
                        trial_id2_updated = f"{trial_id2}{k}_f5"

                        trainer1,trainer2,ds1,ds2,args_init1,args_init2 = load_trainer_ds_from_2_trials(trial_id1_updated,trial_id2_updated,modification = modification,
                                                                                                        model_args=model_args,
                                                                                                        path_model_args=path_model_args,
                                                                                                        ds1_init=ds1,ds2_init=ds2,
                                                                                                        args_init1=args_init1,args_init2=args_init2,
                                                                                                        )
                                                                                                        

                        full_predict1,full_predict2,Y_true,X = get_predict_real_and_inputs(trainer1,trainer2,ds1,ds2,training_mode=training_mode)

                        globals()[f"trainer1_bis{k}"] = trainer1
                        globals()[f"trainer2_bis{k}"] = trainer2
                        globals()[f"ds1_bis{k}"] = ds1
                        globals()[f"ds2_bis{k}"] = ds2
                        globals()[f"full_predict1_bis{k}"] = full_predict1
                        globals()[f"full_predict2_bis{k}"] = full_predict2

                        dic_metric1,dic_metric2 = get_dict_metrics_on_rainy_events(globals()[f"full_predict1_bis{k}"],globals()[f"full_predict2_bis{k}"],Y_true,X,args_init1,args_init2,ds2)

                        # Keep track on metric from model 1
                        if reversed_metric:
                            dic_metric2 = dict(dic_metric1.copy())
                        RMSE = dic_metric2['rmse_all']
                        MAE = dic_metric2['mae_all']
                        MASE = dic_metric2['mase_all']
                        MAPE = dic_metric2['mape_all']
                        if feature_extractor == []:
                            log += f"{model_name}_{target}__e{epochs}_h{horizon}_bis{k}:   All Steps RMSE = {RMSE:.3f}, MAE = {MAE:.3f}, MASE = {MASE:.3f}, MAPE = {MAPE:.3f}\n"
                        else:
                            log += f"{model_name}_{target}_{'_'.join(contextual_data)}_{fusion_strategie}_{feature_extractor}__e{epochs}_h{horizon}_bis{k}:   All Steps RMSE = {RMSE:.3f}, MAE = {MAE:.3f}, MASE = {MASE:.3f}, MAPE = {MAPE:.3f}\n"

                    print(log)


------------------------------------------------------
Contextual data:

Trial ID 1:  STAEformer_bike_out_calendar__e200_h4_bis1_f5
Trial ID 2:  STAEformer_bike_out_weather_calendar_early_fusion_repeat_t_proj__e200_h4_bis1_f5
model_save_path for trial id1:  /home/rrochas/prediction-validation/save/K_fold_validation/training_wo_HP_tuning/Exp2/bike_out_STAEformer/best_models/STAEformer_bike_out_calendar__e200_h4_bis1_f5.pkl
----------------------------------------
Loading the Complete Dataset for K-fold splitting
Coverage Period: 35040 elts between 2019-01-01 00:00:00 and 2019-12-31 23:45:00
Invalid dates within this fold: 0

>>>Tackle Target dataset: bike_out
Loading from /home/rrochas/../../data/rrochas/prediction_validation/agg_data/velov/velov_emitted_by_station15min.csv...
df pivoted:  (70049, 434)
df reindexed :  (35040, 434)
Len coverage period:  35040
df filtered:  (35040, 434)
   Loaded data: (35040, 434)
   Dimension after spatial agg: (35040, 26)
   Init Dataset: 'torch.Size(